# RAG ve Agent Kavramları

## İçerik

1. RAG
2. Embeddings
3. Chunking
4. Vector Store
5. RAG Pipeline
6. Agent
7. Function Calling
8. RAG + Tool Birlikte

## 1. RAG

* **RAG (Retrieval Augmented Generation):** LLM'in eğitim verisi dışındaki bilgilere erişmesini sağlayan mimaridir.
* **Sorun:** LLM'ler sadece eğitim sırasında gördükleri bilgiyi bilir; güncel veya özel dokümanlara erişemez.
* **Çözüm:** Sorgu geldiğinde ilgili parçaları retrieve edip prompt'a context olarak ekleriz; LLM bu context ile yanıt üretir.
* **Akış:** Sorgu → Retrieve → Context ekle → LLM yanıt

## 2. Embeddings

* **Embedding:** Metni sayısal vektöre dönüştürme. Benzer anlamlı metinler vektör uzayında birbirine yakın olur.

Gemini API ile örnek:

In [ ]:
# pip install -q google-genai python-dotenv faiss-cpu
# GEMINI_API_KEY .env dosyasında olmalı

In [1]:
import os
from dotenv import load_dotenv
from google import genai

load_dotenv()
client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

EMBED_MODEL = 'gemini-embedding-001'

In [2]:
soru = "Kredi risk skoru nasıl hesaplanır?"
cevaplar = [
    "Kredi risk skoru 0-1 arası bir değerdir. Bankalar bu skoru kredi onay sürecinde kullanır.",
    "Kredi başvurusu için kimlik, gelir belgesi ve ikametgah gerekir.",
    "Bugün hava çok güzel."
]
# Soru + cevaplar birlikte embed edilir (vecs[0]=soru, vecs[1..3]=cevaplar)
texts = [soru] + cevaplar
result = client.models.embed_content(model=EMBED_MODEL, contents=texts)
print(f"Vektör boyutu: {len(result.embeddings[0].values)}")

Vektör boyutu: 3072


### Benzerlik Ölçümü

* **Neden kosinüs benzerliği?** Embedding'de vektörün yönü (anlam) önemli, uzunluğu değil. Kosinüs sadece yönü ölçer; uzun/kısa metin karşılaştırmasında adil sonuç verir. Skor 0–1 arası (1 = aynı yön).
* **Diğer yöntemler:** Euclidean (L2) mesafesi, nokta çarpımı (dot product), Manhattan (L1). RAG'da kosinüs veya normalize edilmiş dot product yaygındır.

In [3]:
# Soru ile her cevap arasındaki benzerlik: RAG'da en yüksek benzerlik = en ilgili cevap
import math

def cosine_sim(a, b):
    dot = sum(x * y for x, y in zip(a, b))
    norm_a = math.sqrt(sum(x*x for x in a))
    norm_b = math.sqrt(sum(x*x for x in b))
    return dot / (norm_a * norm_b)

vecs = [e.values for e in result.embeddings]
soru_vec = vecs[0]
for i, cevap in enumerate(cevaplar):
    sim = round(cosine_sim(soru_vec, vecs[i + 1]), 4)
    ozet = cevap[:55] + "..." if len(cevap) > 55 else cevap
    print(f"Soru vs Cevap {i+1}: {sim}  |  {ozet}")

Soru vs Cevap 1: 0.8662  |  Kredi risk skoru 0-1 arası bir değerdir. Bankalar bu sk...
Soru vs Cevap 2: 0.6719  |  Kredi başvurusu için kimlik, gelir belgesi ve ikametgah...
Soru vs Cevap 3: 0.5242  |  Bugün hava çok güzel.


## 3. Chunking

* **Chunking:** Uzun metni anlamlı parçalara bölme. Her parça embed edilip vector store'a yazılır.
* **Chunk size:** Çok küçük = bağlam kaybı | çok büyük = gürültü, maliyet.
* **Overlap:** Parçalar arası örtüşme, cümle bölünmesini azaltır.



In [5]:
def simple_chunk(text: str, chunk_size: int = 500, overlap: int = 100) -> list[str]:
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        if chunk.strip():
            chunks.append(chunk.strip())
        start = end - overlap
    return chunks

sample = """Kredi risk skoru, bir bireyin ya da kurumun finansal yükümlülüklerini zamanında ve eksiksiz yerine getirme olasılığını ölçmek amacıyla kullanılan istatistiksel bir göstergedir. Temel olarak, geçmiş finansal davranışlara bakarak gelecekteki ödeme performansına dair bir tahmin üretir. Skor yükseldikçe temerrüt (ödeyememe) olasılığı azalır; skor düştükçe bankalar açısından risk artar. Bu nedenle kredi risk skoru, finansal sistemde hem kredi veren hem de kredi talep eden taraf için kritik bir rol oynar.

Kredi risk skoru hesaplanırken birçok farklı veri noktası dikkate alınır. Bunlar arasında kredi ve kredi kartı ödeme geçmişi, mevcut borçluluk düzeyi, kredi kullanım oranı, yeni açılan kredi hesapları, kredi geçmişinin uzunluğu ve farklı kredi türlerine sahip olma durumu gibi faktörler yer alır. Özellikle ödeme geçmişi, skor üzerinde en yüksek etkiye sahip unsurlardan biridir. Gecikmeli ödemeler, idari ve yasal takibe düşme gibi durumlar kredi skorunu ciddi şekilde olumsuz etkileyebilir. Buna karşılık düzenli ve disiplinli ödeme alışkanlığı, zaman içinde skorun yükselmesini sağlar.

Bankalar ve finansal kuruluşlar, kredi başvurularını değerlendirirken kredi risk skorunu temel bir karar kriteri olarak kullanır. Ancak skor tek başına belirleyici değildir; gelir düzeyi, mevcut borçların toplam gelire oranı, teminat durumu ve başvurulan kredi türü gibi ek unsurlar da değerlendirme sürecine dahil edilir. Yine de kredi risk skoru, ön değerlendirme aşamasında hızlı ve standartlaştırılmış bir analiz imkânı sunduğu için sürecin en önemli bileşenlerinden biridir. Yüksek kredi skoru, genellikle daha uygun faiz oranları ve daha esnek kredi koşulları anlamına gelirken; düşük skor, kredi talebinin reddedilmesine veya daha yüksek maliyetli kredi tekliflerine yol açabilir.

Türkiye’de kredi riskinin izlenmesi ve finansal sistemin sağlıklı işlemesi açısından düzenleyici ve veri sağlayıcı kurumlar önemli rol oynar. Bankacılık Düzenleme ve Denetleme Kurumu (BDDK), bankacılık sektörünü düzenleyen ve denetleyen otorite olarak risk yönetimi çerçevelerini belirler. Bireysel kredi notlarının oluşturulmasında ise Kredi Kayıt Bürosu (KKB) önemli bir veri altyapısı sağlar. KKB bünyesinde faaliyet gösteren sistemler aracılığıyla bireylerin kredi ve kredi kartı ödeme performansları kayıt altına alınır ve finansal kuruluşlarla paylaşılır. Bu yapı, bankalar arasında bilgi asimetrisini azaltarak daha sağlıklı kredi tahsis kararları alınmasına yardımcı olur.

Sonuç olarak kredi risk skoru, hem bireylerin finansal itibarını temsil eden bir gösterge hem de bankalar için risk yönetiminin temel araçlarından biridir. Finansal disiplin, düzenli ödeme alışkanlığı ve dengeli borçlanma stratejisi, uzun vadede kredi skorunun iyileşmesini sağlar. Bu da bireylerin ve kurumların finansal sistem içerisindeki güvenilirliğini artırarak daha avantajlı finansman imkanlarına erişimini mümkün kılar."""

chunk_size, overlap = 500, 110
chunks = simple_chunk(sample, chunk_size=chunk_size, overlap=overlap)
for i, c in enumerate(chunks):
    print(f"Chunk {i+1}: {c}")

# Overlap: Ardışık chunk'lar arasında paylaşılan kısım (son chunk'ın son overlap karakteri = sonraki chunk'ın ilk overlap karakteri)
for i in range(len(chunks) - 1):
    ov = chunks[i][-overlap:]  # Chunk i'nin son overlap karakteri
    print(f"Overlap (Chunk {i+1} -> {i+2}): {ov}")

Chunk 1: Kredi risk skoru, bir bireyin ya da kurumun finansal yükümlülüklerini zamanında ve eksiksiz yerine getirme olasılığını ölçmek amacıyla kullanılan istatistiksel bir göstergedir. Temel olarak, geçmiş finansal davranışlara bakarak gelecekteki ödeme performansına dair bir tahmin üretir. Skor yükseldikçe temerrüt (ödeyememe) olasılığı azalır; skor düştükçe bankalar açısından risk artar. Bu nedenle kredi risk skoru, finansal sistemde hem kredi veren hem de kredi talep eden taraf için kritik bir rol oy
Chunk 2: denle kredi risk skoru, finansal sistemde hem kredi veren hem de kredi talep eden taraf için kritik bir rol oynar.

Kredi risk skoru hesaplanırken birçok farklı veri noktası dikkate alınır. Bunlar arasında kredi ve kredi kartı ödeme geçmişi, mevcut borçluluk düzeyi, kredi kullanım oranı, yeni açılan kredi hesapları, kredi geçmişinin uzunluğu ve farklı kredi türlerine sahip olma durumu gibi faktörler yer alır. Özellikle ödeme geçmişi, skor üzerinde en yüksek etkiye sahip unsurl

## 4. Vector Store

* **Vector Store:** Embedding'leri saklayıp benzerlik araması yapan veritabanı.
* **FAISS:** Hızlı, ölçeklenebilir, Windows/Linux/macOS'ta sorunsuz çalışır.

### Veri Yükleme

`data/soru_cevap_data.md` dosyasından soru-cevap chunk'larını parse eder (`KRD-XXX | S: ... C: ...` formatı).

In [2]:
import re
from pathlib import Path

def load_qa_docs(path: str = "data/soru_cevap_data.md") -> list[str]:
    p = Path(path)
    if not p.exists():
        p = Path("../") / path
    text = p.read_text(encoding="utf-8")
    pattern = r"\*\*KRD-\d+\s*\|\s*S:\s*(.+?)\*\*\s*\nC:\s*(.+?)(?=\n\n\*\*KRD-|\n\n---|\Z)"
    matches = re.findall(pattern, text, re.DOTALL)
    return [f"S: {q.strip()}\nC: {a.strip()}" for q, a in matches if q.strip() and a.strip()]

docs = load_qa_docs()
print(f"{len(docs)} soru-cevap chunk yüklendi.")

300 soru-cevap chunk yüklendi.


### Index Kaydetme / Yükleme

* **Ne yapar:** Dokümanları embed edip FAISS index oluşturur. Index varsa diskten yükler (tekrar API çağrısı yok), yoksa oluşturup `index_store/` klasörüne kaydeder.
* **`.bin`:** FAISS'ın binary formatı. Vektör index'ini kompakt ve hızlı kaydetmek için kullanılır.
* **`.pkl`:** Python pickle. `documents` listesini (soru-cevap metinleri) diske yazmak için, index sadece vektör ID döndürür, metin eşlemesi için gerekli.

In [3]:
import numpy as np
import faiss
import time
import pickle
from pathlib import Path

BATCH_SIZE = 100
WAIT_SEC = 30
INDEX_DIR = Path("data/index_store")
INDEX_PATH = INDEX_DIR / "faiss_index.bin"
DOCS_PATH = INDEX_DIR / "documents.pkl"

def embed_documents(texts: list[str], batch_size: int = BATCH_SIZE) -> np.ndarray:
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        for attempt in range(5):
            try:
                res = client.models.embed_content(model=EMBED_MODEL, contents=batch)
                embeddings.extend([e.values for e in res.embeddings])
                break
            except Exception as e:
                if "429" in str(e) and attempt < 4:
                    time.sleep(WAIT_SEC)
                else:
                    raise
        if i + batch_size < len(texts):
            time.sleep(WAIT_SEC)
    return np.array(embeddings, dtype=np.float32)

def build_faiss_index(documents: list[str]):
    vecs = embed_documents(documents)
    faiss.normalize_L2(vecs)
    index = faiss.IndexFlatIP(vecs.shape[1])
    index.add(vecs)
    return index, documents

def load_or_build_index(documents: list[str]):
    for base in [Path("."), Path("..")]:
        idx_path = base / INDEX_PATH
        docs_path = base / DOCS_PATH
        if idx_path.exists() and docs_path.exists():
            index = faiss.read_index(str(idx_path))
            with open(docs_path, "rb") as f:
                docs = pickle.load(f)
            print(f"Index yüklendi: {len(docs)} doküman ({idx_path})")
            return index, docs

    index, documents = build_faiss_index(documents)
    INDEX_DIR.mkdir(exist_ok=True)
    faiss.write_index(index, str(INDEX_PATH))
    with open(DOCS_PATH, "wb") as f:
        pickle.dump(documents, f)
    print(f"{len(documents)} doküman FAISS index'e eklendi, {INDEX_PATH} kaydedildi.")
    return index, documents

index, documents = load_or_build_index(docs)

Index yüklendi: 300 doküman (..\data\index_store\faiss_index.bin)


### Sorgu Araması

Sorguyu embed edip FAISS'tan top-k en benzer chunk'ları getirir.

In [5]:
def search_similar(query: str, k: int = 5) -> list[str]:
    q_emb = client.models.embed_content(model=EMBED_MODEL, contents=[query])
    q = np.array([q_emb.embeddings[0].values], dtype=np.float32)
    faiss.normalize_L2(q)
    _, indices = index.search(q, k=min(k, len(documents)))
    return [documents[i] for i in indices[0] if i >= 0]

query = "Sen kimsin?"
results = search_similar(query, k=5)
print("Sorgu:", query)
print("Bulunan:")
for doc in results:
    print(" -", doc)

Sorgu: Sen kimsin?
Bulunan:
 - S: Eğitmen kimdir?
C: Eğitmen Enes Fehmi Manan'dır. LinkedIn, GitHub (github.com/enesmanan) ve X (x.com/enesfehmimanan) üzerinden ulaşılabilir.
 - S: Platformu kullanmak için üyelik gerekli mi?
C: Temel bilgilendirme ve chatbot desteği için kayıt gerekmez. Kişiselleştirilmiş analiz için temel bilgi girişi yapılması istenir.
 - S: Kredi başvurusunda nüfus cüzdanı fotokopisi yeterli mi?
C: Güncel kimlik belgesi gereklidir. Yeni tip TC kimlik kartı veya pasaport kabul edilir.
 - S: Bu repo kime ait?
C: Repo Enes Fehmi Manan tarafından Turkish AI Community organizasyonunda inşaa edilmiştir. GitHub adresi github.com/Turkish-AI-Community/hsd-agu-egitim şeklindedir.
 - S: Yabancı uyruklu kişiler kredi alabilir mi?
C: Türkiye'de oturma izni ve çalışma izni olan yabancı uyruklu kişiler kredi başvurusunda bulunabilir. İkamet tezkeresi, gelir belgesi ve pasaport fotokopisi istenir.


## 5. RAG Pipeline

Sorgu → Embed → Top-k chunk al → Prompt'a context ekle → LLM yanıt

Context chunk'ları prompt'a ekleyip LLM'e yollar; bağlamda yoksa "bilmiyorum" der.

In [6]:
from google.genai import types

MODEL = 'gemini-2.5-flash'

def rag_query(question: str, top_k: int = 5) -> str:
    context_chunks = search_similar(question, k=top_k)
    context = "\n".join(context_chunks)
    prompt = f"""Aşağıdaki bağlamı kullanarak soruyu yanıtla. Bağlamda yoksa "bilmiyorum" de.
    Eğer bir yazım yanlışı varsa en mantıklı şekilde düzeltmeye çalış. Cevabında sadece sorulan soruya odaklan, bağlamdaki bilgileri kullanarak mümkün olduğunca kısa ve net yanıt ver. Gereksiz detaylardan kaçın.
eğer inşaa kelimesi varsa bunu inşa olarak düzeltmeye çalış.
BAĞLAM:
{context}

SORU: {question}

YANIT:"""
    response = client.models.generate_content(
        model=MODEL,
        contents=prompt,
        config=types.GenerateContentConfig(temperature=0.2, max_output_tokens=2048)
    )
    return "".join(p.text for p in response.candidates[0].content.parts if p.text)

print(rag_query("Bu uygulama kim tarafından yapılmıştır?"))

Enes Fehmi Manan tarafından Turkish AI Community organizasyonunda inşa edilmiştir.


## 6. Agent

* **Agent:** LLM + araçlar (tools). Model ne zaman hangi tool'u çağıracağına karar verir.
* **Agent loop:** Mesaj al → Tool gerekli mi? → Evet: tool çağır, sonucu modele ver → Hayır: direkt yanıt ver

## 7. Function Calling

* **Function Calling:** Model uygun gördüğünde fonksiyonu çağırır; siz sonucu modele geri verirsiniz.
* Gemini API: `Tool` ve `function_declarations` ile desteklenir.

### Tool Tanımı

`get_risk_level(skor)`: 0-1 arası skor alır, risk seviyesi (düşük/orta/yüksek) döner.

In [7]:
# Örnek: get_risk_level - 0-1 arası skor alır, risk seviyesi döner
get_risk_level_declaration = types.FunctionDeclaration(
    name="get_risk_level",
    description="0-1 arası kredi risk skorunu alır, risk seviyesini (düşük/orta/yüksek) döner",
    parameters={
        "type": "object",
        "properties": {
            "skor": {"type": "number", "description": "0-1 arası risk skoru"}
        },
        "required": ["skor"]
    }
)

def get_risk_level(skor: float) -> str:
    if skor < 0.33: return "yüksek"
    if skor < 0.66: return "orta"
    return "düşük"

tools = types.Tool(function_declarations=[get_risk_level_declaration])

### Function Call Örneği

Model prompt'a göre tool çağırırsa sonucu yazdırır; yoksa direkt metin yanıtı verir.

In [8]:
def run_with_tool(prompt: str) -> str:
    config = types.GenerateContentConfig(tools=[tools], temperature=0)

    # 1) Send user prompt to model
    response = client.models.generate_content(model=MODEL, contents=prompt, config=config)
    part = response.candidates[0].content.parts[0]

    # If model doesn't request a tool, return its text directly
    if not (hasattr(part, "function_call") and part.function_call):
        return "".join(p.text for p in response.candidates[0].content.parts if hasattr(p, "text") and p.text)

    # 2) Model requested a tool call -- execute the function
    fc = part.function_call
    args = dict(fc.args) if fc.args else {}
    result = get_risk_level(args.get("skor", 0))
    print(f"Tool call: {fc.name}({args}) -> {result}")

    # 3) Send FunctionResponse back to model so it can compose a natural-language answer
    followup = client.models.generate_content(
        model=MODEL,
        config=config,
        contents=[
            types.Content(role="user", parts=[types.Part.from_text(text=prompt)]),
            types.Content(role="model", parts=[part]),
            types.Content(role="user", parts=[
                types.Part.from_function_response(name=fc.name, response={"result": result})
            ]),
        ],
    )
    return "".join(p.text for p in followup.candidates[0].content.parts if hasattr(p, "text") and p.text)

print(run_with_tool("Skorum 0.25 çıktı, risk seviyem ne?"))

Tool call: get_risk_level({'skor': 0.25}) -> yüksek
Risk seviyeniz yüksek.


## 8. RAG + Tool Birlikte (Agentic RAG)

* **Agentic RAG:** Hem dokümanlardan bilgi çeker (RAG) hem de API/tool çağırır (Agent).
* Örnek: "Kredi skoru nedir?" → RAG context ile yanıt; "Skorum 0.25, risk seviyem ne?" → `get_risk_level` tool çağrısı
* System prompt `system_instruction` üzerinden verilir, RAG context'i ve tool bilgisi burada tanımlanır.

### Agentic RAG Örneği

In [9]:
def agentic_rag(user_message: str, top_k: int = 5, max_turns: int = 5) -> str:
    context = "\n".join(search_similar(user_message, k=top_k))

    system_prompt = f"""Sen kredi risk danışmanısın. Aşağıdaki bağlamı kullan. Bağlamda yoksa "bilmiyorum" de.
Tool'un var: get_risk_level(skor) - 0-1 arası skor alır, risk seviyesi (düşük/orta/yüksek) döner.
Skor ile ilgili sorularda bu tool'u kullan.

BAĞLAM:
{context}"""

    config = types.GenerateContentConfig(
        system_instruction=system_prompt,
        tools=[tools],
        temperature=0.2,
        max_output_tokens=2048,
    )
    contents = [types.Content(role="user", parts=[types.Part.from_text(text=user_message)])]

    for _ in range(max_turns):
        response = client.models.generate_content(model=MODEL, contents=contents, config=config)
        parts = response.candidates[0].content.parts

        fc_part = next((p for p in parts if hasattr(p, "function_call") and p.function_call), None)
        if fc_part is None:
            return "".join(p.text for p in parts if hasattr(p, "text") and p.text)

        fc = fc_part.function_call
        args = dict(fc.args) if fc.args else {}
        result = get_risk_level(args.get("skor", 0))

        contents.append(types.Content(role="model", parts=[fc_part]))
        contents.append(types.Content(
            role="user",
            parts=[types.Part.from_function_response(name=fc.name, response={"result": result})]
        ))

    return "Max turn aşıldı."

print("--- RAG sorusu ---")
print(agentic_rag("XGBoost nedir?"))
print("\n--- Tool sorusu ---")
print(agentic_rag("Skorum 0.25 çıktı, risk seviyem ne?"))

--- RAG sorusu ---
XGBoost, gradient boosting tabanlı güçlü bir ensemble algoritmadır. Sıralı olarak ağaçlar ekleyerek hatayı minimize eder. Hızlı, performanslı ve düzenlileştirme destekli olması nedeniyle kredi skorlamada sıklıkla tercih edilir.

--- Tool sorusu ---
Skorunuz 0.25 ise risk seviyeniz yüksek.
